# **Install bayesian-optimization Library**

In [1]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=b7b3ec46e51094fad44dfa933dda2c6588dff146380458ee3f2d8d36ee86ed26
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


# **Install gpy gpyopt Library**

In [2]:
pip install gpy gpyopt

     |████████████████████████████████| 1.0MB 6.7MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
  Created wheel for gpy: filename=GPy-1.9.9-cp36-cp36m-linux_x86_64.whl size=2633933 sha256=514230514d2704bbec53ab470f74c29882c93f38bbb1357ac2fb417e36b7de55
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for gpyopt: filename=GPyOpt-1.2.6-cp36-none-any.whl size=83623 sha256=d04c9d5547892e691cdfd80fd874780ed4c22cccea1ceb7503415b9920af72c3
  Stored in directory: /root/.cache/pip/wheels/b2/00/69/cfa967a125cf25e66f644be6193ad6f0edf231147879ad714f
  Created wheel for paramz: filename=paramz-0.9.5-cp36-none-any.whl size=102552 sha256=6a80cb9a89892618ff6bffa96661ecb802240a4dbe35bad713d255310a316cbb
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built gpy gpyopt paramz


# **Import necessary Libraries**

In [3]:
import GPy, GPyOpt
import numpy as np
import pandas as pds
import random
from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# **MNIST class -  functions for data loading, training, fitness calculation and evaluation**

In [4]:
# MNIST class
class MNIST():
    def __init__(self,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 bn1=0,
                 bn2=0,
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.bn1 = bn1
        self.bn2 = bn2
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.mnist_data()
        self.__model = self.mnist_model()
        
    # load mnist data from keras dataset
    def mnist_data(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        Y_train = np_utils.to_categorical(y_train, 10)
        Y_test = np_utils.to_categorical(y_test, 10)
        return X_train, X_test, Y_train, Y_test
    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(784,)))
        if self.bn1 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        if self.bn2 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def evaluate_mnist(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

# **Run the model for MNIST**

In [6]:

# function to run mnist class
def run_mnist(l1_out=512, l2_out=512, 
              l1_drop=0.2, l2_drop=0.2, 
              bn1=0, bn2=0,
              batch_size=100, epochs=10, validation_split=0.1):
    
    _mnist = MNIST()
    mnist_evaluation = _mnist.evaluate_mnist()
    return mnist_evaluation

# **Obtaining the bounds for hyperparameters**

In [7]:
# bounds for hyper-parameters in mnist model
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_drop',          'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l2_drop',          'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_out',           'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)},
          {'name': 'l2_out',           'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)},
          {'name': 'bn1',              'type': 'discrete',    'domain': (0, 1)},
          {'name': 'bn2',              'type': 'discrete',    'domain': (0, 1)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (10, 100, 500)},
          {'name': 'epochs',           'type': 'discrete',    'domain': (5, 10, 20)}]

# **Bayesian Optimization Implementation**

In [8]:
# function to optimize mnist model
def f(x):
    print(x)
    evaluation = run_mnist(
        l1_drop = int(x[:,1]), 
        l2_drop = int(x[:,2]), 
        l1_out = float(x[:,3]),
        l2_out = float(x[:,4]), 
        bn1 = int(x[:,5]),
        bn2 = int(x[:,6]),
        batch_size = int(x[:,7]), 
        epochs = int(x[:,8]), 
        validation_split = float(x[:,0]))
    print("loss:{0} \t\t accuracy:{1}".format(evaluation[0], evaluation[1]))
    return evaluation[0]

# **Instance for Optimizer**

In [9]:
# optimizer
opt_mnist = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

[[1.17182991e-01 1.32565514e-01 2.86688817e-01 5.12000000e+02
  1.28000000e+02 1.00000000e+00 0.00000000e+00 5.00000000e+02
  1.00000000e+01]]
11493376/11490434 [==============================] - 0s 0us/step
Epoch 00005: early stopping
loss:0.06382651627063751 		 accuracy:0.9793999791145325
[[2.56116831e-02 1.30016176e-01 1.50512844e-01 1.28000000e+02
  5.12000000e+02 0.00000000e+00 0.00000000e+00 5.00000000e+02
  2.00000000e+01]]
Epoch 00006: early stopping
loss:0.07548926025629044 		 accuracy:0.9765999913215637
[[2.74490436e-01 1.16512479e-01 2.75954016e-01 1.28000000e+02
  1.28000000e+02 0.00000000e+00 1.00000000e+00 1.00000000e+02
  5.00000000e+00]]
Epoch 00008: early stopping
loss:0.06443511694669724 		 accuracy:0.9814000129699707
[[1.86705664e-01 2.06677203e-01 1.08750486e-01 1.28000000e+02
  5.12000000e+02 1.00000000e+00 1.00000000e+00 1.00000000e+02
  5.00000000e+00]]
Epoch 00005: early stopping
loss:0.07538910955190659 		 accuracy:0.9778000116348267
[[2.26998270e-02 1.39335094

# **Run the optimizer**

In [10]:
# optimize mnist model
opt_mnist.run_optimization(max_iter=15)

[[1.08845670e-01 1.02703193e-01 9.02670597e-02 6.40000000e+01
  1.28000000e+02 0.00000000e+00 1.00000000e+00 5.00000000e+02
  1.00000000e+01]]
Epoch 00005: early stopping
loss:0.0743269994854927 		 accuracy:0.9776999950408936
[[8.60897233e-03 1.07679352e-01 7.40544129e-02 6.40000000e+01
  1.28000000e+02 0.00000000e+00 0.00000000e+00 5.00000000e+02
  1.00000000e+01]]
Epoch 00003: early stopping
loss:0.08681721985340118 		 accuracy:0.9735999703407288
[[3.88434817e-02 1.16884732e-01 2.83535178e-01 5.12000000e+02
  1.28000000e+02 1.00000000e+00 0.00000000e+00 5.00000000e+02
  1.00000000e+01]]
Epoch 00003: early stopping
loss:0.0843592956662178 		 accuracy:0.9740999937057495
[[1.21474907e-01 5.25444509e-02 2.16087269e-02 1.28000000e+02
  1.28000000e+02 1.00000000e+00 1.00000000e+00 1.00000000e+02
  5.00000000e+00]]
Epoch 00004: early stopping
loss:0.07485908269882202 		 accuracy:0.9783999919891357
[[  0.17234373   0.18179626   0.16872517  64.          64.
    1.           0.         100.   

# **Output Generation**

In [11]:
optbounds = {
    'validation_split' : opt_mnist.x_opt[0],
    'l1_drop' : opt_mnist.x_opt[1],
    'l2_drop' : opt_mnist.x_opt[2],
    'l1_out' : opt_mnist.x_opt[3],
    'l2_out' : opt_mnist.x_opt[4],
    'bn1' : opt_mnist.x_opt[5],
    'bn2' : opt_mnist.x_opt[6],
    'batch_size' : opt_mnist.x_opt[7],
    'epochs' : opt_mnist.x_opt[8],
}

# **The optimized hyperparameters**

In [12]:
# print optimized mnist model
print("optimized parameters: {0}".format(optbounds))
print("optimized loss: {0}".format(opt_mnist.fx_opt))

optimized parameters: {'validation_split': 0.022699826989121173, 'l1_drop': 0.1393350944111121, 'l2_drop': 0.12644504812304083, 'l1_out': 64.0, 'l2_out': 128.0, 'bn1': 0.0, 'bn2': 0.0, 'batch_size': 500.0, 'epochs': 10.0}
optimized loss: 0.06211815029382706
